In [8]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

client = None

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import prompts.discounts.block_schema
from importlib import reload
import utils

reload(prompts.discounts.block_schema)
reload(utils)


from prompts.discounts.block_schema import correct_block_schema, generate_different_scenarious_by_decision_points, \
    generate_thread_metadata_by_scenario, company_system_message, example_scenarios_V1, \
    get_fields_for_scneraios_from_block_schema, RESULT_BLOCK_SCHEMA, generated_scenarios_v1, \
    continue_generating_different_scenarious_by_decision_points, generated_scenarios_without_points_v2, \
    example_scenarios_V2

from utils import get_scenarios


print('Updated')

Updated


In [126]:
def create_completion(messages, temperature, output = True, model: str = 'gpt-4', **kwargs):    
    global client
    if os.getenv('OPENAI_API_KEY') is None:
        load_dotenv()

    if client is None:
        client = openai.OpenAI()

    response = client.chat.completions.create(model=model, messages=messages, 
                                              temperature=temperature, stream=True, **kwargs)
    collected_messages = []
    for chunk in response:
        if chunk.choices[0].delta.content:
            if output:
                print(chunk.choices[0].delta.content, end='')
            collected_messages.append(chunk.choices[0].delta.content)

    return ''.join(collected_messages)
    

In [114]:
message_with_request = "Witam ZAPYTANIE ODNOŚNIE DOSTĘPNOŚCI N/W ARTYKUŁÓW - Prowadnik kabli -- Cable carrier ( prowadnik kabla ) 1665.030.200.3000-4655 TSO_0 FA_MA Tsubaki 1szt Energy chain (prowadnik kabla ) 1665.030.125.140-4189.5 TS0_0 FA_MA Tsubaki 1szt Jęśli możecie dostarczyć proszę o ofertę Pozdrawiam / Best regards Adam Janura Kom. / Mobile +48 537-797-300 www.landoia.pl \"LANDOIA\" Kapusta Łukasz 26-613 Radom, ul. Marii Gajl 1 NIP 7962440697"

message_with_discount = "Hello Due to the fact that our client has accepted our offer for the device which includes your Tsubaki cable guides, please confirm that the offer is valid. An additional discount from you will be appreciated. A quick response will allow us to place an order with your company for the offered items."

email = message_with_discount

In [84]:
    # Tasks:
    #     - Classify intent of email message
    #       Input: { "email_message": string }
    #     - Find most suitable intent for target intent
    #       Input: { "target_intent": string, "intents": array(string)}

system_prompt = """
You are Senior Sales manager of first line of communication with customer. When customer send message you are the first person that define intents from email
and decide what to do next 

You have in under the authority following sales managers:
- Discount Sales manager: takes tasks related with negotiations abount discounts, can accept only messages from customers with discount intent 
- Intent Classification Manager: helps to manage work with intents
- Customer Service Manager: this sales manager could answer on any customer questions, that other sales cannot to do.

"""

prompt = """
As a Senior Sales Manager of first line of service, please read the following email from customer, exract all intents and then decide what 
sales manager you would delegate the task to write message. Do it step by step.

Messages processing instruction:
1. First of all you need to get message intent to better understand which manager could handle the email at next.
2. When you got the intent you needed to pass email to manager that has competence to work with message with intents.

[EMAIL FROM CUSTOMER]
{message_with_discount}
[/EMAIL FROM CUSTOMER]

Allowed keys from storage: {task_inputs}

Thought: you should always think about what to do
Assignee: the assignee manager from list
Task: the task for assignee manager
Task Input: take allowed key from storage that would needed to task completion or none, for instance it could look like ['intents', 'email']
Observation: the result of the task from manager
... (this Thought/Assignee/Task/Task Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""



In [234]:
import agents.senior_sales.agent

reload(agents.senior_sales.agent)

from agents.senior_sales.agent import SeniorSalesAgent

In [235]:
senior_sales = SeniorSalesAgent()

In [236]:
senior_sales.run(email, **{
    'intents': intents
})

Senior Sales

Thought: I need to define the intents in this email to understand which manager can handle it.

Assignee: Intent Classification Manager
Task: Define the intents in the customer's email.
Task Input: ['email']



Intent classifier

Thought: To define the intents in the customer's email, I need to analyze the email content and classify it based on the intents.
Instruction: Intent Classification
Instruction Input: ['email', 'intents']
```json
{
    "intents": [
        {
            "intent": "Order Processing",
            "sub_intent": "Order Confirmation",
            "branch": "Confirmation of Specific Offer/Product Order"
        },
        {
            "intent": "Pricing Inquiries and Confirmation",
            "sub_intent": "Pricing Inquiries",
            "branch": "Discount Requests"
        }
    ]
}
```


Thought: The intents in the customer's email have been successfully classified.
Final Answer: The intents in the customer's email are: Order Processing, Order Co

"Thought: Based on the identified intents, the Discount Sales Manager is competent to handle this email because it involves negotiation about discounts.\nAssignee: Discount Sales Manager\nTask: Respond to the customer's email addressing the intents identified: Order Processing, Order Confirmation, Confirmation of Specific Offer/Product Order, Pricing Inquiries and Confirmation, Pricing Inquiries, and Discount Requests.\nTask Input: ['intents', 'email']"

In [87]:
def senior_sales_manager(email: str, **kwargs):
    kwargs['email'] = email

    # print(f"[{', '.join(kwargs.keys())}]")
    # print(prompt.format(message_with_discount=email, task_inputs=', '.join(kwargs.keys()))+ '\n\n')
    
    response = create_completion([
        { "role": "assistant", "content": system_prompt},
        { "role": "user", "content": prompt.format(message_with_discount=email, task_inputs=f"[{', '.join(kwargs.keys())}]")}
    ], 
    stop=['\nObservation:', '\n\tObservation:'],                        
    temperature=0.3)

    return response

In [88]:
response = senior_sales_manager(message_with_discount)

Thought: This customer email seems to have two primary intents. The first one is a request for confirmation that the offer for the Tsubaki cable guides is still valid. The second one is a negotiation for an additional discount on the products. 

Assignee: Intent Classification Manager
Task: Classify the intents present in the customer email.
Task Input: ['email']


In [90]:
import re
import json

regex = (
            r"Assignee\s*\d*\s*:[\s]*(.*?)[\s]*Task\s*\d*\s*:[\s]*(.*)Task\s*\d*\s*Input:[\s]*(.*)"
        )
action_match = re.search(regex, response, re.DOTALL)
       
    
action_match.group(1), action_match.group(2), action_match.group(3)

('Intent Classification Manager',
 'Classify the intents present in the customer email.\n',
 "['email']\n")

In [93]:
import ast

assignee = action_match.group(1)
task = action_match.group(2)
task_input = ast.literal_eval(action_match.group(3))

# task_input[0]

In [ ]:
if assignee == 'Intent Classification Manager':
    

### Intent Classifier

In [116]:
itent_classifier_main_prompt = """
Please do the task from higher Sales Manager.
The task: {task}

The available instructions:
1. Instruction: Intent Classification
Insruction Input: ['email', 'intents']
2. Instruction: Find the most related intents

Allowed keys from storage: email, intents

Use the following format:

Thought: you should always think about what to do
Instruction: the intruction to take, should be one of available instructions
Instruction Input: take allowed key from storage that would needed to instruction completion or none, 
for instance it could look like ['intents', 'email']
Observation: the result of the action
... (this Thought/Instruction/Instruction Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""

In [117]:
response = create_completion([
            # { "role": "assistant", "content": system_prompt},
            { "role": "user", "content": itent_classifier_main_prompt.format(task=task)}
], 
stop=['\nObservation:', '\n\tObservation:'],                            
temperature=0.3)


Thought: To classify the intents in the customer email, I should use the 'Intent Classification' instruction.
Instruction: Intent Classification
Instruction Input: ['email', 'intents']


In [ ]:
response = create_completion([
    { "role": "assistant", "content": system_prompt},
    { "role": "user", "content": prompt}
], temperature=0.7)

In [118]:
def parse_instruction(response):
    regex = (r"Instruction\s*\d*\s*:[\s]*(.*?)[\s]*Instruction\s*\d*\s*Input:[\s]*(.*)")
    
    instruction_match = re.search(regex, response, re.DOTALL)
    
    return instruction_match.group(1), ast.literal_eval(instruction_match.group(2))

instruction, instruction_input = parse_instruction(response)
instruction, instruction_input

('Intent Classification', ['email', 'intents'])

In [119]:
intents = """
# Consolidated Intents Hierarchy

## Order Processing
- Order Confirmation
  - Confirmation of Specific Offer/Product Order
  - Confirmation of Order Receipt and Acceptance
- Order Documentation and Information
  - Providing and Requesting Order-Related Information

## Payment Processing and Confirmation
- Invoice Processing
- Payment Confirmation
  - Confirming and Inquiring about Payment Methods

## Delivery and Shipping
- Delivery and Shipping Confirmation
  - Confirmation of Shipping Details
  - Pickup and Delivery Method Information
- Delivery Logistics Discussion
  - Discussing Shipping and Self-Collection Options

## Pricing Inquiries and Confirmation
- Transport and Delivery Pricing
  - Inquiries and Updates on Transport Costs
- Pricing Inquiries
  - Discount Requests
  - Quotation Requests
  - Price Confirmations

## Product Information and Availability
- Product Details
  - Information on Product Portfolio and Availability

## Sales and Customer Service Follow-up
- Follow-up on Quotations
- Customer Service Assistance

"""

In [115]:
kwargs = {
    'email': email,
    'intents': intents
}

In [121]:
INTENT_FORMAT = """
Put the result at ```json``` format, like:
{
    "intents": [
        {
            "intent": "<main intent>",
            "sub_intent": "<sub intent>",
            "branch": "<branch>"
        }
    ]
}
"""

intent_classification_prompt = """
Please classifiy for email message [EMAIL MESSAGE] all intent to better classification and search. You can use our default list of intents [INTENTS] or
if there is no suitable intent you can generate your own.

[EMAIL MESSAGE]
{email}
[/EMAIL MESSAGE]

[INTENTS]
{intents}
[/INTENTS]

For example, this message would have intents:
- customer: Hello So as not to waste precious time. We order devices from offer no. 440822 of friday, 29 september 2023 . 
If possible, please provide an additional discount on your purchase. Please confirm acceptance of our order. 
In the meantime, once you know what the transport cost will be, we will discuss how to deliver the parcel to us 
or whether we will collect it ourselves.

Example of intents/sub-intents/branches namings:
   - Order Processing -> Order Placement Confirmation -> Confirmation of Specific Offer
   - Order Processing -> Order Acceptance Confirmation -> Confirmation of Order Receipt and Acceptance
   - Pricing and Quotes -> Discount Inquiry -> Request for Additional Discount on Purchase
   - Pricing and Quotes -> Transport Cost Inquiry -> Inquiry About Delivery Costs
   - Delivery and Shipping -> Delivery Method Discussion -> Discussing Whether to Ship or Self-Collect
   - Delivery and Shipping -> Discussing Logistics -> Coordination of Transport and Delivery Options

Response format:
{intent_response_format}
"""

In [ ]:
print(intent_classification_prompt.format(intent_response_format=INTENT_FORMAT, **kwargs))

In [125]:
response = create_completion([
    { "role": "user", "content": intent_classification_prompt.format(intent_response_format=INTENT_FORMAT, **kwargs)}
], temperature=0.7)

{
    "intents": [
        {
            "intent": "Order Processing",
            "sub_intent": "Order Confirmation",
            "branch": "Confirmation of Specific Offer/Product Order"
        },
        {
            "intent": "Pricing Inquiries and Confirmation",
            "sub_intent": "Pricing Inquiries",
            "branch": "Discount Requests"
        }
    ]
}

In [200]:
import agents.intent_classifier.classify_intents
import agents.intent_classifier.agent

reload(agents.intent_classifier.classify_intents)
reload(agents.intent_classifier.agent)

from agents.intent_classifier.classify_intents import ClassifyIntentsInstruction
from agents.intent_classifier.agent import IntentClassifierAgent


In [201]:
agent = IntentClassifierAgent()

In [202]:
agent.run(task, **{
    'email': email,
    'intents': intents
})

Thought: I need to classify the intents present in the customer email.
Instruction: Intent Classification
Instruction Input: ['email', 'intents']```json
{
    "intents": [
        {
            "intent": "Order Processing",
            "sub_intent": "Order Confirmation",
            "branch": "Confirmation of Specific Offer/Product Order"
        },
        {
            "intent": "Pricing Inquiries and Confirmation",
            "sub_intent": "Pricing Inquiries",
            "branch": "Discount Requests"
        }
    ]
}
```Thought: The intents have been successfully classified.
Final Answer: The intents present in the customer email are Order Processing, Order Confirmation, Confirmation of Specific Offer/Product Order, Pricing Inquiries and Confirmation, Pricing Inquiries, and Discount Requests.

'The intents present in the customer email are Order Processing, Order Confirmation, Confirmation of Specific Offer/Product Order, Pricing Inquiries and Confirmation, Pricing Inquiries, and Discount Requests.'

In [136]:
intent_classifier = ClassifyIntentsInstruction()

In [139]:
ClassifyIntentsInstruction.instruction

'\nPlease classifiy for email message [EMAIL MESSAGE] all intent to better classification and search. You can use our default list of intents [INTENTS] or\nif there is no suitable intent you can generate your own.\n\n[EMAIL MESSAGE]\n{email}\n[/EMAIL MESSAGE]\n\n[INTENTS]\n{intents}\n[/INTENTS]\n\nFor example, this message would have intents:\n- customer: Hello So as not to waste precious time. We order devices from offer no. 440822 of friday, 29 september 2023 . \nIf possible, please provide an additional discount on your purchase. Please confirm acceptance of our order. \nIn the meantime, once you know what the transport cost will be, we will discuss how to deliver the parcel to us \nor whether we will collect it ourselves.\n\nExample of intents/sub-intents/branches namings:\n   - Order Processing -> Order Placement Confirmation -> Confirmation of Specific Offer\n   - Order Processing -> Order Acceptance Confirmation -> Confirmation of Order Receipt and Acceptance\n   - Pricing and Q

In [138]:
r = intent_classifier.run(**{
    'email': email,
    'intents': intents
})

print(r)

{
    "intents": [
        {
            "intent": "Order Processing",
            "sub_intent": "Order Confirmation",
            "branch": "Confirmation of Specific Offer/Product Order"
        },
        {
            "intent": "Pricing Inquiries and Confirmation",
            "sub_intent": "Pricing Inquiries",
            "branch": "Discount Requests"
        }
    ]
}Intents:
Order Processing -> Order Confirmation -> Confirmation of Specific Offer/Product Order
Pricing Inquiries and Confirmation -> Pricing Inquiries -> Discount Requests
